In [50]:
require('tidyverse')
require('data.table')
options(stringsAsFactors=FALSE)
## TODO - need this BY VISIT!

hormone_drugs <- read.delim("../data/hormone_drugs_todownload.txt")
head(hormone_drugs)


code,drug,read_codes,drugbank_id,hormone
"G03AC05, G03DB02",megestrol,1140884706,DB00351,progesterone
G03XB01,mifepristone,1140869112;1141157302,DB00834,progesterone
"G03BA02, G03EK01",methyltestosterone,1140857656,DB06710,testosterone
G04CB01,finasteride,1140868550,DB01216,testosterone
G03AC09,desogestrel,1140857912;1141182794,DB00304,progesterone
G03DB01,dydrogesterone,1140868494,DB00378,progesterone


In [51]:
hdrugs2 <-separate_rows(hormone_drugs, read_codes, sep=";")
hdrugs2$trait <- sapply(hdrugs2$read_codes, function(x) sprintf("MED%s", x))
hdrugs3 <- filter(hdrugs2, trait != "MED")

In [4]:
phe_loc <- fread("/oak/stanford/groups/mrivas/ukbb/24983/phenotypedata/most_recent_phenos_20190422.txt", header=FALSE, data.table=FALSE)


Loading required package: data.table

Attaching package: ‘data.table’

The following objects are masked from ‘package:dplyr’:

    between, first, last

The following object is masked from ‘package:purrr’:

    transpose



In [5]:
colnames(phe_loc) <- "path"
phe_loc$phe_id <- sapply(phe_loc$path, function(x) {y <- strsplit(x, "/")[[1]]; strsplit(y[[length(y)]], ".", fixed=TRUE)[[1]][[1]]})


In [52]:
hdrug_paths <- left_join(hdrugs3, phe_loc, by=c("trait"="phe_id"))
head(hdrug_paths)
table(is.na(hdrug_paths$path)) # woot! all present

code,drug,read_codes,drugbank_id,hormone,trait,path
"G03AC05, G03DB02",megestrol,1140884706,DB00351,progesterone,MED1140884706,/oak/stanford/groups/mrivas/ukbb24983/phenotypedata/9796/24611/phe/MED1140884706.phe
G03XB01,mifepristone,1140869112,DB00834,progesterone,MED1140869112,/oak/stanford/groups/mrivas/ukbb24983/phenotypedata/9796/24611/phe/MED1140869112.phe
G03XB01,mifepristone,1141157302,DB00834,progesterone,MED1141157302,/oak/stanford/groups/mrivas/ukbb24983/phenotypedata/9796/24611/phe/MED1141157302.phe
"G03BA02, G03EK01",methyltestosterone,1140857656,DB06710,testosterone,MED1140857656,/oak/stanford/groups/mrivas/ukbb24983/phenotypedata/9796/24611/phe/MED1140857656.phe
G04CB01,finasteride,1140868550,DB01216,testosterone,MED1140868550,/oak/stanford/groups/mrivas/ukbb24983/phenotypedata/9796/24611/phe/MED1140868550.phe
G03AC09,desogestrel,1140857912,DB00304,progesterone,MED1140857912,/oak/stanford/groups/mrivas/ukbb24983/phenotypedata/9796/24611/phe/MED1140857912.phe



FALSE 
   37 

In [29]:
getDrugEntries <- function(path, name){
    df <- fread(path, header=FALSE, sep=" ", data.table=FALSE)
    df2 <- filter(df, V3==2)
    colnames(df2) <- c("IID", "IID2", name)
    print(sprintf("%s patients taking %s", nrow(df2), name))
    return(select(df2,-IID2))
}



In [31]:
drug_dat <- lapply(1:nrow(hdrug_paths), function(i) getDrugEntries(hdrug_paths$path[[i]], hdrug_paths$trait[[i]]))


[1] "18 patients taking MED1140884706"
[1] "0 patients taking MED1140869112"
[1] "0 patients taking MED1141157302"
[1] "1 patients taking MED1140857656"
[1] "1977 patients taking MED1140868550"
[1] "48 patients taking MED1140857912"
[1] "6 patients taking MED1141182794"
[1] "29 patients taking MED1140868494"
[1] "3 patients taking MED1140868526"
[1] "333 patients taking MED1141168574"
[1] "181 patients taking MED1140868588"
[1] "3 patients taking MED1141181124"
[1] "10 patients taking MED1141181218"
[1] "21 patients taking MED1141181220"
[1] "14 patients taking MED1141166196"
[1] "0 patients taking MED1141180988"
[1] "4 patients taking MED1141179820"
[1] "18 patients taking MED1140909848"
[1] "401 patients taking MED1141192000"
[1] "25 patients taking MED1140869366"
[1] "16 patients taking MED1141157410"
[1] "65 patients taking MED1140869270"
[1] "246 patients taking MED1141181594"
[1] "300 patients taking MED1140868580"
[1] "18 patients taking MED1141157406"
[1] "0 patients taking MED

In [63]:
num_patients <- lapply(drug_dat, nrow)
names(num_patients) <- hdrug_paths$trait
hdrug_paths$num <- num_patients
hdrug_paths2 <- filter(hdrug_paths, num > 0)
head(select(hdrug_paths2, c(drug, drugbank_id, trait, num)))
# COLLAPSE BY DRUGBANK ID 
length(unique(hdrug_paths2$drugbank_id)) # 25 instead of 37
length(unique(hdrug_paths2$drug))
trait_to_drug <- split(hdrug_paths2$drug, hdrug_paths2$trait)
drug_to_trait <- split(hdrug_paths2$trait, hdrug_paths2$drug)
mult_map <- drug_to_trait[sapply(drug_to_trait, length) > 1]

drug,drugbank_id,trait,num
megestrol,DB00351,MED1140884706,18
methyltestosterone,DB06710,MED1140857656,1
finasteride,DB01216,MED1140868550,1977
desogestrel,DB00304,MED1140857912,48
desogestrel,DB00304,MED1141182794,6
dydrogesterone,DB00378,MED1140868494,29


[1] 25

[1] 25

In [64]:
complete_drug_dat <- drug_dat[num_patients > 0] %>% reduce(full_join, by = "IID")
complete_drug_dat[is.na(complete_drug_dat)] <- 0
complete_drug_dat[complete_drug_dat==2] <- 1

In [86]:
collapse_cols <- sapply(drug_to_trait, function(x) ifelse(length(x)==1, data.frame(complete_drug_dat[,x]),data.frame(apply(complete_drug_dat[,x], 1, sum))))

In [88]:
collapse_df <- (do.call(cbind, collapse_cols))

In [93]:
drug_dat <- cbind(complete_drug_dat$IID, collapse_df) 
head(drug_dat)
colnames(drug_dat)[1] <- "IID"

,clomifene,cyproterone,danazol,desogestrel,diethylstilbestrol,drospirenone,dutasteride,dydrogesterone,estradiol,⋯,levonorgestrel,medroxyprogesterone acetate,megestrol,mesterolone,methyltestosterone,norethisterone,progesterone,raloxifene,testosterone,tibolone
5246110,0,0,0,0,0,0,0,0,0,⋯,0,0,1,0,0,0,0,0,0,0
4263041,0,0,0,0,0,0,0,0,0,⋯,0,0,1,0,0,0,0,0,0,0
2224100,0,0,0,0,0,0,0,0,0,⋯,0,0,1,0,0,0,0,0,0,0
1242351,0,0,0,0,0,0,0,0,0,⋯,0,0,1,0,0,0,0,0,0,0
1787587,0,0,0,0,0,0,0,0,0,⋯,0,0,1,0,0,0,0,0,0,0
1300650,0,0,0,0,0,0,0,0,0,⋯,0,0,1,0,0,0,0,0,0,0


In [95]:
apply(drug_dat, 2, sum)
write.table(drug_dat, file="../data/id_to_hormone_drug.txt", row.names=FALSE, quote=FALSE, sep="\t")

IID                   clomifene 
                20820511511                           3 
                cyproterone                     danazol 
                         47                          18 
                desogestrel          diethylstilbestrol 
                         54                          18 
               drospirenone                 dutasteride 
                          4                         401 
             dydrogesterone                   estradiol 
                         29                         987 
                    estriol            ethinylestradiol 
                        246                          31 
               etonogestrel                   etynodiol 
                         14                           3 
                finasteride         hydroxyprogesterone 
                       1977                           2 
             levonorgestrel medroxyprogesterone acetate 
                         41                          65 
                  megestrol                 mesterolone 
                         18                           3 
         methyltestosterone              norethisterone 
                          1                         318 
               progesterone                  raloxifene 
                        181                         333 
               testosterone                    tibolone 
                        346                         876